In [ ]:
import os
import pandas as pd
import geopandas as gpd
import unidecode

import plotly.express as px

In [ ]:
# Define a pasta base (diretório onde está este script)
BASE_DIR = os.getcwd()

# Caminhos dos arquivos
FILE_PATH_DATASET = os.path.join(BASE_DIR, 'datasets', 'RECLAMEAQUI_HAPVIDA.csv')
#FILE_PATH_SHP = os.path.join(BASE_DIR, 'datasets', 'BR_Municipios_2024.shp')

# O arquivo final será o mesmo tratado
FILE_PATH_OUTPUT = os.path.join(BASE_DIR, 'datasets', 'RECLAMEAQUI_HAPVIDA_treat.csv')

In [ ]:
def load_data(file_path):
    #Carrega o dataset original.
    
    df = pd.read_csv(file_path, sep=',', encoding='utf-8')
    return df

def split_city_state(df, column='LOCAL'):
    # Divide a coluna "LOCAL" no formato 'Cidade - UF' em duas colunas separadas: 'Cidade' e 'UF'.
    
    if column in df.columns:
        split_cols = df[column].str.split(' - ', expand=True)
        df['Cidade'] = split_cols[0]
        df['UF'] = split_cols[1]
        df.drop(columns=[column], inplace=True)
    return df

def padronizar_nome(texto):
    #Padroniza nomes removendo acentos e deixando em maiúsculo.
    
    return unidecode.unidecode(str(texto)).strip().upper()

In [ ]:
# 1) Leitura do CSV original
print(f"Lendo arquivo: {FILE_PATH_DATASET}")
df = load_data(FILE_PATH_DATASET)
print(f"Dataset carregado com {df.shape[0]} linhas e {df.shape[1]} colunas.")
dff = df.copy()

# Dividindo coluna LOCAL por CIDADE e ESTADO.
dff[['CIDADE', 'ESTADO']] = dff['LOCAL'].str.split(' - ', expand=True)

# Gerando tamanho do texto
dff['TAMANHO_TEXTO'] = dff['DESCRICAO'].str.len()

# removendo duas linas que não tem estado
dff = dff[~(dff['ESTADO'] == '--')]

dff.head()

In [ ]:
dff.info()

# Figure Serie Temporal

In [ ]:
import processamento


fig_serie_temporal = px.line(
        #dff.groupby('MES').size().reset_index(name='CONTAGEM'), 
        processamento.DataProcessing(df=df).series_temporal(),
        x='MES', 
        y='CONTAGEM',
        title='Série Temporal de Reclamações', 
        labels={'MES': 'Mês', 'CONTAGEM': 'Nº Reclamações'}
    ).update_layout(title_x=0.5, margin=dict(t=50, l=10, r=10, b=10))

fig_serie_temporal.show()

# Figure reclamações por estado

In [ ]:
fig_estado_data = px.bar(
        #dff['ESTADO'].value_counts().reset_index(),
        processamento.DataProcessing(df=df).data_estado(),
        x='ESTADO', 
        y='count', 
        title='Reclamações por Estado', 
        labels={'ESTADO': 'Estado', 'count': 'Nº Reclamações'}, 
        text_auto=True
    ).update_xaxes(categoryorder="total descending").update_layout(title_x=0.5, margin=dict(t=50, l=10, r=10, b=10))

fig_estado_data.show()

# Figure distribuição por status

In [ ]:
fig_status_data = px.pie(
        #dff['STATUS'].value_counts().reset_index(),
        processamento.DataProcessing(df=df).data_status(),
        names='STATUS', 
        values='count', 
        title='Distribuição por Status', 
        hole=0.4
    ).update_layout(title_x=0.5, margin=dict(t=50, l=10, r=10, b=10))

fig_status_data.show()

# Figure distribuição tamanho do texto

In [ ]:

fig_dist_texto = px.histogram(    
        #dff, 
        processamento.DataProcessing(df=df).data_texto(),
        x='TAMANHO_TEXTO', 
        title='Distribuição do Tamanho do Texto',
        labels={'TAMANHO_TEXTO': 'Tamanho do Texto', 'count': 'Frequência'}
    ).update_layout(title_x=0.5, margin=dict(t=50, l=10, r=10, b=10))
fig_dist_texto.show()

# Figure mapa de reclamações

In [ ]:
# Para completar os estados que não tem dados no dataframe
 
dict_estados = {
    'AC': 'Acre', 
    'AL': 'Alagoas', 
    'AP': 'Amapá', 
    'AM': 'Amazonas', 
    'BA': 'Bahia',
    'CE': 'Ceará',
    'DF': 'Distrito Federal', 
    'ES': 'Espírito Santo', 
    'GO': 'Goiás',
    'MA': 'Maranhão', 
    'MT': 'Mato Grosso', 
    'MS': 'Mato Grosso do Sul', 
    'MG': 'Minas Gerais',
    'PA': 'Pará', 
    'PB': 'Paraíba', 
    'PR': 'Paraná', 
    'PE': 'Pernambuco', 
    'PI': 'Piauí',
    'RJ': 'Rio de Janeiro',
    'RN': 'Rio Grande do Norte', 
    'RS': 'Rio Grande do Sul',
    'RO': 'Rondônia', 
    'RR': 'Roraima', 
    'SC': 'Santa Catarina', 
    'SP': 'São Paulo',
    'SE': 'Sergipe', 
    'TO': 'Tocantins'
}

ano_mapa = 2022
# Salva um dataframe para tratamento dos dados

df_contagem_estado = dff[dff['ANO'] == ano_mapa].groupby('ESTADO').size().reset_index(name='CONTAGEM')

# Adiciona a silga e contagem em zero do estado que não tem no dataframe

for sigla in dict_estados.keys():
    if sigla not in set(df_contagem_estado['ESTADO'].unique()):
        df_contagem_estado.loc[len(df_contagem_estado)] = {'ESTADO': sigla, 'CONTAGEM': 0}

# Adiciona dos nomes dos estados.
#         
df_contagem_estado['NOME'] = df_contagem_estado['ESTADO'].map(dict_estados)
df_contagem_estado

In [ ]:
import json
import processamento

ano_mapa = 2022

#caminho_arquivo = './datasets/brasil_estados.json'

#with open(caminho_arquivo, 'r', encoding='utf-8') as f:
#    dados = json.load(f)      
    

fig_map_data = px.choropleth_mapbox(
    #dff[dff['ANO'] == ano_mapa].groupby('ESTADO').size().reset_index(name='CONTAGEM'),
    #df_contagem_estado,
    #geojson=dados,
    processamento.DataProcessing(df=df).data_mapa(ano_mapa=ano_mapa),
    geojson=processamento.DataProcessing.open_geo_json(),
    locations='ESTADO',     
    color='CONTAGEM', 
    color_continuous_scale="reds",
    mapbox_style="carto-positron", 
    zoom=3.2, 
    center={"lat": -14.2350, "lon": -51.9253},
    title=f'Reclamações em {ano_mapa}', 
    labels={'CONTAGEM': 'Nº Reclamações'}
).update_layout(title_x=0.5, margin=dict(t=50, l=0, r=0, b=0))

fig_map_data.show()


In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import matplotlib.pyplot as plt
from wordcloud import WordCloud 

# Obter a lista de stopwords em português
stopwords_portugues = stopwords.words('portuguese')

stopwords_planos_saude = {
    # Termos Gerais de Planos de Saúde
    'plano', 'planos', 'saúde', 'convênio', 'convênios', 'seguro', 'seguros',
    'operadora', 'operadoras', 'benefício', 'benefícios', 'serviço', 'serviços',
    'contrato', 'contratos', 'apólice', 'apólices', 'cobertura', 'coberturas',
    'rede', 'redes', 'credenciada', 'credenciadas', 'atendimento', 'atendimentos',
    'assistência', 'assistências', 'médico', 'médica', 'médicos', 'médicas',
    'hospital', 'hospitais', 'clínica', 'clínicas', 'exame', 'exames', 'consulta',
    'consultas', 'procedimento', 'procedimentos', 'reembolso', 'reembolsos',
    'carência', 'carências', 'mensalidade', 'mensalidades', 'custo', 'custos',
    'preço', 'preços', 'valor', 'valores', 'pagamento', 'pagamentos', 'titular',
    'dependentes', 'adesão', 'portabilidade', 'urgência', 'emergência', 'ans',
    'agência', 'nacional',

    # Termos Relacionados/Descritivos Comuns - podem não ser o foco
    'cliente', 'clientes', 'usuário', 'usuários', 'paciente', 'pacientes',
    'doutor', 'doutora', 'doutores', 'doutoras', 'equipe', 'equipes',
    'empresa', 'empresas', 'filho', 'filha', 'vc',    
    'bom', 'boa', 'bons', 'boas', 'ruim', 'ruins', 'ótimo', 'ótima',
    'excelente', 'péssimo', 'péssima', 'muito', 'pouco', 'mais', 'menos',
    'assim', 'porém', 'entanto', 'também', 'ainda', 'já', 'sempre', 'nunca',
    'alguns', 'algumas', 'todo', 'toda', 'todos', 'todas', "pois", "outro", 
    "outra", "dia", "dias", "entrega", "reclame", "aqui", "problema",
    'q', 'fiz', ',', 'Hapvida', "não", 'nao', "pra", "tive", "minha", 'contato',
    'fazer', 'nada', 'ter', 'preciso', 'ano', 'onde', 'vai', 'após',
    'hoje', 'SAC', 'vez', 'pode', 'lá', 'por que', 'porque','pq', 'faz', 'vou',
    'quero', 'disse', 'data', 'então', 'ir', 'desde', 'sendo', 'agora',
    'meses', 'entrei', 'consegui', 'passar', 'pessoa', 'entrar', 'semana',
    'sobre', 'horário', 'hora', 'mês', 'min', 'ja', 'mesma', 'pago', 'passou',
    'bem', 'atendente', 'bem', 'anos', 'liguei', 'mail', 'saber', 'ligar',
    'quase', 'mim', 'apenas','muito', 'pouco', 'mais', 'menos', 'bastante', 
    'quase', 'tão', 'tão', 'demais', 'suficiente', 'suficientes', 'totalmente', 
    'parcialmente', 'completamente', 'absolutamente', 'relativamente', 
    'excessivamente', 'apenas', 'só','bom', 'boa', 'bons', 'boas', 'ruim', 'ruins', 
    'excelente', 'excelentes', 'péssimo', 'péssima', 'péssimos', 'péssimas',
    'melhor', 'melhores', 'pior', 'piores', 'grande', 'grandes', 'pequeno', 'pequena',
    'pequenos', 'pequenas', 'certo', 'certa', 'certos', 'certas', 'errado', 'errada',
    'errados', 'erradas', 'positivo', 'positiva', 'positivos', 'positivas',
    'negativo', 'negativa', 'negativos', 'negativas', 'legal', 'legais',
    'incrível', 'incríveis', 'horrível', 'horríveis', 'razoável', 'razoáveis',
    'bom', 'má', 'maus', 'más', 'grande', 'pequeno', 'novo', 'nova', 'novos', 
    'novas', 'velho', 'velha', 'velhos', 'velhas', 'ótimo', 'ótima', 'ótimos', 'ótimas',    
    'sempre', 'nunca', 'jamais', 'às vezes', 'frequentemente', 'raramente', 'constantemente',
    'diariamente', 'semanalmente', 'mensalmente', 'anualmente', 'hoje', 'ontem', 'amanhã',
    'agora', 'depois', 'antes', 'cedo', 'tarde', 'logo', 'já', 'ainda', 'durante',    
    'assim', 'bem', 'mal', 'melhor', 'pior', 'rapidamente', 'lentamente', 'facilmente',
    'dificilmente', 'geralmente', 'especialmente', 'principalmente', 'claro', 'clara',
    'claramente', 'verdadeiro', 'verdadeira', 'verdadeiros', 'verdadeiras', 'falso',
    'falsa', 'falsos', 'falsas', 'possível', 'possíveis', 'impossível', 'impossíveis',    
    'porém', 'contudo', 'entretanto', 'todavia', 'assim', 'portanto', 'logo', 'daí',
    'além', 'ainda', 'inclusive', 'mesmo', 'embora', 'apesar', 'conforme', 'segundo',
    'exceto', 'salvo', 'inclusive', 'além disso', 'em vez de', 'ao invés de', 'ou seja',    
    'aqui', 'lá', 'ali', 'cá', 'perto', 'longe', 'dentro', 'fora', 'em cima', 'em baixo',
    'atrás', 'frente', 'lado', 'ao lado', 'direita', 'esquerda', 'meio',    
    'próprio', 'própria', 'próprios', 'próprias', 'outro', 'outra', 'outros', 'outras',
    'mesmo', 'mesma', 'mesmos', 'mesmas', 'todo', 'toda', 'todos', 'todas',
    'algum', 'alguma', 'alguns', 'algumas', 'nenhum', 'nenhuma', 'nenhuns', 'nenhumas',
    'vários', 'várias', 'diversos', 'diversas', 'certo', 'certa', 'certos', 'certas',
    'tal', 'tais', 'qualquer', 'quaisquer', 'cujo', 'cuja', 'cujos', 'cujas',
    'primeiro', 'primeira', 'segundo', 'segunda', 'último', 'última',
    'único', 'única', 'únicos', 'únicas', 'sistema', 'atende', 'momento', 'telefone',
    'vezes'
}

for palavra in stopwords_planos_saude:
    stopwords_portugues.append(palavra)

In [ ]:
texto = " ".join(dff['DESCRICAO'].astype(str).tolist())
print(len(texto)) 
print(texto[:100])  

In [ ]:
wordcloud = WordCloud(
    width=800,
    height=400,
    background_color='white',
    stopwords=stopwords_portugues,
    colormap='viridis', 
    max_words=50
).generate(texto)

In [ ]:

plt.figure(figsize=(12, 6))
#plt.imshow(wordcloud, interpolation='bilinear')
plt.imshow(processamento.DataProcessing(df=df).data_wordcloud(), interpolation='bilinear')
plt.axis('off')
plt.title('WordCloud - Termos mais frequentes nas reclamações')
plt.show()